In [ ]:
import re
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional
import glob
import sklearn
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Subset,DataLoader
import torchvision.transforms as transforms
import torchvision
from torch.autograd import Variable
import random
from google.colab import files
from sklearn.metrics import  confusion_matrix
from sklearn.model_selection import ShuffleSplit
import cv2
from google.colab.patches import cv2_imshow
from scipy.ndimage import distance_transform_edt
from torch.autograd import Variable
import skimage.segmentation
import skimage.io
import skimage
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import skimage.io
from skimage import feature
from skimage import filters
import copy
import torchvision
from collections import OrderedDict
import math
import torch
import scipy
from scipy import signal as signal
import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA, PCA
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy.signal import ellip, lfilter
from scipy.signal import freqz
import matplotlib.pyplot as plt

In [ ]:
!pip install torch_geometric
import torch_geometric.nn as gnn
from torch_geometric.data import Data
from torch_geometric.data import DataLoader as DataLoader_graph
from torch_geometric.nn import global_mean_pool,global_max_pool,ChebConv

In [ ]:
#dataset_parameters

dataset_parameter = dict(
    fs = 250,
    channel = 22,
    num_classes = 4,
    num_subject = 10
)

In [ ]:
#model_parameters

parameter = dict(
N = 5000,
F = 1024,
P = 784,
H = 28,
W = 28,
kH = 9,
kW =9,
kH1 = 9,
kW1 = 9,
C11 = 32,
C12 = 8,
C3 =16,
C_FC1 = 512,
C_FC2 = 1024,
m_plus = 0.9,
m_minus = 0.1,
alpha = 0.0001,
lamda = 0.5,
batch = 4
)

In [ ]:
#data_loading

dframe = pd.DataFrame
ns = dataset_parameter['num_subject']
ori_data1 = dict()
mod_data1 = dict()
def subjectCounter(i):
    return 'subject0{}'.format(i)

for i in range(1, ns):
    data_path1 = '/content/A0'+str(i)+'T.npz'.format(i)
    subj = subjectCounter(i)
    ori_data1[subj] = np.load(data_path1)
    mod_data1[subj] = {}
    mod_data1[subj]['raw_EEG'] = ori_data1[subj]['s']
pd.DataFrame(mod_data1['subject01']['raw_EEG']).head()
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data1[subj]['raw_EEG'] = np.delete(mod_data1[subj]['raw_EEG'], np.s_[22:], 1)
    print(mod_data1['subject01']['raw_EEG'].shape)

#data_filtering

lowcut=4
highcut=40
fs = dataset_parameter['fs']

def elliptical_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    ripple = 0.001
    attenuation = 100
    low = lowcut/nyq
    high = highcut/nyq
    b,a = ellip(order, ripple, attenuation, [low, high], btype='band')
    y = lfilter(b, a, signal)
    return y

def elliptical_bandpass_one_subject(data, subj, lowcut, highcut, fs, interval=None):
    print('Processing ', subj)
    data[subj]['EEG_filtered'] = {}
    temp_raw_EEG = data[subj]['raw_EEG']
    if interval is not None:
        startband = np.arange(lowcut, highcut, step = interval)
        for start in startband:
            band = "{:02d}_{:02d}".format(start, start+interval)
            print('Filtering through {} Hz band'.format(band))
            data[subj]['EEG_filtered'][band] = {}
            data[subj]['EEG_filtered'][band]['EEG_all'] = elliptical_bandpass_filter(temp_raw_EEG, start, start+interval, fs)
    else:
        band = "{:02d}_{:02d}".format(lowcut, highcut)
        data[subj]['EEG_filtered'][band]['EEG_all'] = elliptical_bandpass_filter(temp_raw_EEG, lowcut, highcut, fs)

for i in range(1, ns):
    subj = subjectCounter(i)
    elliptical_bandpass_one_subject(mod_data1, subj, lowcut, highcut, fs, interval=36)
    print('')

#seggregating data class-wise

left_pos = 'left_pos'
right_pos = 'right_pos'
tongue_pos = 'tongue_pos'
feet_pos = 'feet_pos'
start = 0.5
end = 3.5
EEG_left = 'EEG_left'
EEG_right = 'EEG_right'
EEG_tongue = 'EEG_tongue'
EEG_feet = 'EEG_feet'
for i in range(1, ns):
  subj = subjectCounter(i)
  mod_data1[subj][left_pos] = ori_data1[subj]['epos'][ori_data1[subj]['etyp'] == 769]
  mod_data1[subj][right_pos] = ori_data1[subj]['epos'][ori_data1[subj]['etyp'] == 770]
  mod_data1[subj][tongue_pos] = ori_data1[subj]['epos'][ori_data1[subj]['etyp'] == 772]
  mod_data1[subj][feet_pos] = ori_data1[subj]['epos'][ori_data1[subj]['etyp'] == 771]
for i in range(1, ns):
  subj = subjectCounter(i)
  print('Processing ', subj)
  temp_pos_left1 = mod_data1[subj][left_pos]
  temp_pos_right1 = mod_data1[subj][right_pos]
  temp_pos_tongue1 = mod_data1[subj][tongue_pos]
  temp_pos_feet1 = mod_data1[subj][feet_pos]
  for band in mod_data1[subj]['EEG_filtered'].keys():
    temp_EEG_all = mod_data1[subj]['EEG_filtered'][band]['EEG_all']
    temp_EEG_left1 = []
    temp_EEG_right1 = []
    temp_EEG_tongue1 = []
    temp_EEG_feet1 = []
    for j in range(len(temp_pos_left1)):
        temp_EEG_left1.append(temp_EEG_all[temp_pos_left1[j] + int(start*fs) : temp_pos_left1[j] + int(end*fs),:])
    mod_data1[subj]['EEG_filtered'][band]['EEG_left'] = np.array(temp_EEG_left1)
    for j in range(len(temp_pos_right1)):
        temp_EEG_right1.append(temp_EEG_all[temp_pos_right1[j] + int(start*fs) : temp_pos_right1[j] + int(end*fs),:])
    mod_data1[subj]['EEG_filtered'][band]['EEG_right'] = np.array(temp_EEG_right1)
    for j in range(len(temp_pos_tongue1)):
        temp_EEG_tongue1.append(temp_EEG_all[temp_pos_tongue1[j] + int(start*fs) : temp_pos_tongue1[j] + int(end*fs),:])
    mod_data1[subj]['EEG_filtered'][band]['EEG_tongue'] = np.array(temp_EEG_tongue1)
    for j in range(len(temp_pos_feet1)):
        temp_EEG_feet1.append(temp_EEG_all[temp_pos_feet1[j] + int(start*fs) : temp_pos_feet1[j] + int(end*fs),:])
    mod_data1[subj]['EEG_filtered'][band]['EEG_feet'] = np.array(temp_EEG_feet1)
  mod_data1[subj]['EEG_filtered'][band]

#train_test_data_split

def split_EEG_one_class(EEG_one_class,ratio=0.5):
    nr = int(ratio * np.shape(EEG_one_class)[0])
    EEG_train = EEG_one_class[:nr,:,:]
    EEG_test = EEG_one_class[nr:,:,:]
    return EEG_train, EEG_test

for i in range(1, ns):
    subj = subjectCounter(i)
    temp_EEG_left1 = mod_data1[subj]['EEG_filtered']['04_40']['EEG_left']
    temp_EEG_right1 = mod_data1[subj]['EEG_filtered']['04_40']['EEG_right']
    temp_EEG_tongue1 = mod_data1[subj]['EEG_filtered']['04_40']['EEG_tongue']
    temp_EEG_feet1 = mod_data1[subj]['EEG_filtered']['04_40']['EEG_feet']
    temp_filt = mod_data1
    temp_filt[subj]['EEG_left_train'], temp_filt[subj]['EEG_left_test'] = split_EEG_one_class(temp_EEG_left1,ratio = 0.5)
    temp_filt[subj]['EEG_right_train'], temp_filt[subj]['EEG_right_test'] = split_EEG_one_class(temp_EEG_right1,ratio = 0.5)
    temp_filt[subj]['EEG_tongue_train'], temp_filt[subj]['EEG_tongue_test'] = split_EEG_one_class(temp_EEG_tongue1,ratio = 0.5)
    temp_filt[subj]['EEG_feet_train'], temp_filt[subj]['EEG_feet_test'] = split_EEG_one_class(temp_EEG_feet1,ratio = 0.5)

#Subject specific data

subj = 'subject01'
t_train = np.concatenate((temp_filt[subj]['EEG_left_train'],temp_filt[subj]['EEG_right_train'],temp_filt[subj]['EEG_tongue_train'],temp_filt[subj]['EEG_feet_train']), axis=0)
t_test = np.concatenate((temp_filt[subj]['EEG_left_test'],temp_filt[subj]['EEG_right_test'],temp_filt[subj]['EEG_tongue_test'],temp_filt[subj]['EEG_feet_test']), axis=0)
t_train = np.reshape(t_train, [np.shape(t_train)[0],np.shape(t_train)[2],np.shape(t_train)[1]])
t_test = np.reshape(t_test, [np.shape(t_test)[0],np.shape(t_test)[2],np.shape(t_test)[1]])

p1 = [0]*np.shape(temp_filt[subj]['EEG_left_train'])[0]
p2 = [1]*np.shape(temp_filt[subj]['EEG_right_train'])[0]
p3 = [2]*np.shape(temp_filt[subj]['EEG_tongue_train'])[0]
p4 = [3]*np.shape(temp_filt[subj]['EEG_feet_train'])[0]
label_train = np.concatenate((p1,p2,p3,p4), axis=0)

p1 = [0]*np.shape(temp_filt[subj]['EEG_left_test'])[0]
p2 = [1]*np.shape(temp_filt[subj]['EEG_right_test'])[0]
p3 = [2]*np.shape(temp_filt[subj]['EEG_tongue_test'])[0]
p4 = [3]*np.shape(temp_filt[subj]['EEG_feet_test'])[0]
label_test = np.concatenate((p1,p2,p3,p4), axis=0)

print(np.shape(t_train))
print(np.shape(t_test))
print(np.shape(label_train))
print(np.shape(label_test))

class CustomDataset(Dataset):
    def __init__(self, input_paths, target_paths):

        self.input_paths = input_paths
        self.target_paths = target_paths

    def __getitem__(self, index):
        p = np.shape(self.input_paths[index])[0]
        input = self.input_paths[index]
        input = torch.from_numpy(input)
        classes = torch.from_numpy(np.array(self.target_paths[index]))
        return input,classes

    def __len__(self):
        return len(self.input_paths)

train_dataset = CustomDataset(t_train, label_train)
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=parameter['batch'], shuffle=True, num_workers=2)
test_dataset = CustomDataset(t_test, label_test)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=parameter['batch'], shuffle=False, num_workers=2)
print(len(train_dataset))
print(len(test_dataset))
print(np.shape(train_dataset[0][0]))
print(np.shape(test_dataset[0][0]))

In [ ]:
cuda_enable = 1
device = torch.device("cuda" if cuda_enable else "cpu")

In [ ]:
#defining edges for graph

def edge_idx(x):
  edges=[]
  for i in range(dataset_parameter['channel']):
    for j in range(dataset_parameter['channel']):
      y=[i]
      if i!=j:
        y.append(j)
        edges.append(y)
  edges=torch.tensor(edges)
  return edges
def edge_wt(x,edge):
  weights=torch.zeros(((dataset_parameter['channel'])*(dataset_parameter['channel']-1),2))
  k = torch.zeros((dataset_parameter['channel'])*(dataset_parameter['channel']-1))
  x_mean=torch.mean(x,0,keepdim=True)
  x_std=torch.std(x,0,keepdim=True)
  x=(x-x_mean)/x_std
  for i in range(edge.shape[0]):
    print()
    output=torch.mean((x[edge[i][0],:]*x[edge[i][1],:]))
    weights[i,0]=output
    k[i] = output
  return k

In [ ]:
###graph_model

class graph_model(nn.Module):
  def __init__(self,N,F,batch,height,width):
    super(graph_model,self).__init__()
    self.batch = batch
    self.height = height
    self.width = width
    P = int(height*width)
    self.conv1=gnn.ChebConv(N, F, bool=False, K =2, normalization = None)
    self.conv2=gnn.ChebConv(F, P, bool=False, K = 2, normalization = None)
    self.fc=nn.Linear(P, 1)

  def forward(self, x, edge_index, edge_weight):
    B = torch.tensor([0]*dataset_parameter['channel'])
    x = x.cpu()
    edge_index = edge_index.cpu()
    edge_weight = edge_weight.cpu()
    x = self.conv1(x, edge_index, edge_weight, B)
    x = self.conv2(x, edge_index, edge_weight, B)
    x_out_mean=torch.reshape(gnn.global_mean_pool(x, B),[self.batch,1,self.height,self.width])
    x_out_max=torch.reshape(gnn.global_max_pool(x,B),[self.batch,1,self.height,self.width])
    preds = self.fc(torch.flatten((x_out_mean+x_out_max),start_dim = 1))
    return x_out_mean, x_out_max, torch.sigmoid(preds)

class graph_att(nn.Module):
  def __init__(self,input_features,F,batch,height,width,device):
    super(graph_att,self).__init__()
    self.device = device
    self.graph_block=graph_model(input_features,F,batch,height,width)
    self.batch = batch
    self.height = height
    self.width = width

  def forward(self,x1,x2,x_graph,edge_index,edge_weight):
    x_g_mean,x_g_max,preds=self.graph_block(x_graph,edge_index,edge_weight)
    x_g_mean=torch.reshape(x_g_mean,(self.batch,1,self.height,self.width))
    x_g_max=torch.reshape(x_g_max,(self.batch,1,self.height,self.width))
    x=torch.cat((x1*(x_g_mean.to(self.device)),x2*(x_g_max.to(self.device))),1)
    return x, preds.to(self.device)

In [ ]:
###graph_2d_conv_attention

class ChannelAttention(nn.Module):
    def __init__(self, in_planes):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

    def forward(self, x):
        avg_out = self.avg_pool(x)
        max_out = self.max_pool(x)
        out = avg_out + max_out
        return F.relu(out)

class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = avg_out + max_out
        return F.relu(x)

class attlayer(nn.Module):
    def __init__(self, in_channel):
        super(attlayer, self).__init__()
        self.ca=ChannelAttention(in_planes=in_channel)
        self.sa=SpatialAttention()

    def forward(self, x):
        x = torch.cat((self.ca(x)*x,self.sa(x)*x),1)
        return x

In [ ]:
###decoder_model

def mask(out_digit_caps,device):
    v_length = torch.sqrt((out_digit_caps**2).sum(dim=2))
    _, max_index = v_length.max(dim=1)
    max_index = max_index.data
    batch_size = out_digit_caps.size(0)
    masked_v = [None] * batch_size
    for batch_ix in range(batch_size):
        sample = out_digit_caps[batch_ix]
        v = Variable(torch.zeros(sample.size()))
        v = v.to(device)
        max_caps_index = max_index[batch_ix]
        v[max_caps_index] = sample[max_caps_index]
        masked_v[batch_ix] = v
    masked = torch.stack(masked_v, dim=0)
    return masked

class Decoder(nn.Module):
    def __init__(self, num_classes, output_unit_size, input_height, input_width,num_channel,
                 fc1_output_size, fc2_output_size,device):
        super(Decoder, self).__init__()
        self.device = device
        self.input_height = input_height
        self.input_width = input_width
        self.fc1 = nn.Linear(num_classes * output_unit_size, fc1_output_size).to(self.device)
        self.fc2 = nn.Linear(fc1_output_size, fc2_output_size).to(self.device)
        self.fc3 = nn.Linear(fc2_output_size, int(input_height*input_width)).to(self.device)
        self.conv1 = nn.Conv2d(1,num_channel,kernel_size=3,stride=1,padding=1,bias=False,device=self.device)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, target):
        batch_size = target.size(0)
        masked_caps = mask(x,self.device)
        vector_j = masked_caps.view(x.size(0), -1)
        fc1_out = self.relu(self.fc1(vector_j))
        fc2_out = self.relu(self.fc2(fc1_out))
        fc3_out = self.relu(self.fc3(fc2_out))
        output = torch.reshape(fc3_out,(batch_size,1,self.input_height, self.input_width))
        reconstruction = self.conv1(output)
        return reconstruction

In [ ]:
###capsule_network

def squash(sj, dim=2):
    sj_mag_sq = torch.sum(sj**2, dim, keepdim=True)
    sj_mag = torch.sqrt(sj_mag_sq)
    v_j = (sj_mag_sq / (1.0 + sj_mag_sq)) * (sj / sj_mag)
    return v_j

class CapsuleLayer(nn.Module):
    def __init__(self, in_unit, in_channel, num_unit, unit_size, primary_capsule_channel, kH, kW,  kH1, kW1, use_routing,
                 num_routing,device):
        super(CapsuleLayer, self).__init__()
        self.in_unit = in_unit
        self.in_channel = in_channel
        self.num_unit = num_unit
        self.primary_capsule_channel = primary_capsule_channel
        self.use_routing = use_routing
        self.num_routing = num_routing
        self.device = device
        if self.use_routing:
            self.weight = nn.Parameter(torch.randn(1, in_channel, num_unit, unit_size, in_unit))
        else:
            self.conv0 = nn.Conv2d(self.in_channel,self.num_unit*self.primary_capsule_channel,kernel_size=[kH,kW],stride=1,padding=0,device=self.device)
            self.conv_units = nn.ModuleList([
                nn.Conv2d(self.num_unit*self.primary_capsule_channel, self.primary_capsule_channel, kernel_size=[kH1,kW1], stride=2,padding=0,device=self.device) for u in range(self.num_unit)
            ])

    def forward(self, x):
        if self.use_routing:
            return self.routing(x)
        else:
            return self.no_routing(x)

    def routing(self, x):
        batch_size = x.size(0)
        x = x.transpose(1, 2)
        x = torch.stack([x] * self.num_unit, dim=2).unsqueeze(4)
        batch_weight = torch.cat([self.weight] * batch_size, dim=0)
        u_hat = torch.matmul(batch_weight.to(self.device), x)
        b_ij = Variable(torch.zeros(1, self.in_channel, self.num_unit, 1))
        b_ij = b_ij.to(self.device)
        num_iterations = self.num_routing
        c_ij=Variable(torch.ones((1, self.in_channel, self.num_unit, 1)))
        c_ij = c_ij.to(self.device)
        for iteration in range(num_iterations):
            c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(4)
            s_j = (c_ij * u_hat).sum(dim=1, keepdim=True)
            v_j = squash(s_j, dim=3)
            v_j1 = torch.cat([v_j] * self.in_channel, dim=1)
            u_vj1 = torch.matmul(u_hat.transpose(3, 4), v_j1).squeeze(4).mean(dim=0, keepdim=True)
            b_ij = b_ij + u_vj1
            c_ij = F.mish(b_ij,inplace=False)
        return v_j.squeeze(1)

    def no_routing(self, x):
        x = self.conv0(x)
        unit = [self.conv_units[i](x) for i, l in enumerate(self.conv_units)]
        unit = torch.stack(unit, dim=1)
        batch_size = x.size(0)
        unit = unit.view(batch_size, self.num_unit, -1)
        return squash(unit, dim=2)

In [ ]:
# AIDC-CN

class Net(nn.Module):
    def __init__(self, graph_input_feature,
                 num_conv_channel, num_primary_unit,
                 primary_unit_size, primary_capsule_channel, kH, kW, kH1, kW1, num_classes, output_unit_size, num_routing,
                 input_height,input_width,F,batch, fc1_output_size, fc2_output_size,device):
        super(Net, self).__init__()

        self.device = device
        self.loss_ce=nn.CrossEntropyLoss().to(self.device)
        self.num_classes = num_classes
        self.conv0 = nn.Conv2d(in_channels=num_conv_channel,
                               out_channels=num_conv_channel,
                               kernel_size=3,
                               stride=1,padding=1,device=self.device)
        self.self_att = attlayer(num_conv_channel)
        self.graph_att =  graph_att(graph_input_feature,F,batch,input_height,input_width,self.device)
        self.primary = CapsuleLayer(in_unit=0,
                                    in_channel=4*num_conv_channel,
                                    num_unit=num_primary_unit,
                                    unit_size=primary_unit_size,
                                    primary_capsule_channel = primary_capsule_channel,
                                    kH = kH, kW = kW, kH1 = kH1, kW1 = kW1,
                                    use_routing=False,
                                    num_routing=num_routing,
                                    device=self.device)
        self.digits = CapsuleLayer(in_unit=num_primary_unit,
                                   in_channel=primary_unit_size,
                                   num_unit=num_classes,
                                   unit_size=output_unit_size,
                                   primary_capsule_channel = 0,
                                   kH = 0, kW = 0, kH1 = 0, kW1 = 0,
                                   use_routing=True,
                                   num_routing=num_routing,
                                   device=self.device)
        self.decoder = Decoder(num_classes, output_unit_size,
                                   input_height, input_width, num_conv_channel, fc1_output_size, fc2_output_size, self.device)

    def forward(self, x1, edge_index, edge_weight, x2, label):
        out_conv = F.relu(self.conv0(x2))
        out_conv1 = self.self_att(out_conv)
        out_conv11 = F.relu(self.conv0(x2))
        out_conv12 = F.relu(self.conv0(x2))
        out_conv2, preds_graph = self.graph_att(out_conv11,out_conv12, x1, edge_index, edge_weight)
        input_caps=torch.cat((out_conv1,out_conv2), 1)
        out_primary_caps = self.primary(input_caps)
        out_digit_caps = self.digits(out_primary_caps)
        label1 = torch.cat((label,torch.tensor([i for i in range(1, self.num_classes)]).to(self.device)))
        label2=F.one_hot(label1.to(torch.int64),num_classes=self.num_classes)
        target=label2[:self.num_classes]
        recon=self.decoder(out_digit_caps,target)
        return out_digit_caps,recon,preds_graph,target

    def loss(self, out_digit_caps, graph_preds, recon, input_img, target, label,alpha,m_plus,m_minus,loss_lambda, size_average=True):
        m_loss = self.margin_loss(out_digit_caps, target,m_plus,m_minus,loss_lambda)
        rec_loss = self.recloss(recon, input_img)
        if size_average:
            m_loss = m_loss.mean()
            rec_loss = rec_loss.mean()
        graph_loss=self.loss_ce(graph_preds, label.unsqueeze(1))
        return alpha*m_loss + rec_loss + graph_loss

    def margin_loss(self, input, target,m_plus,m_minus,loss_lambda):
        batch_size = input.size(0)
        v_c = torch.sqrt((input**2).sum(dim=2, keepdim=True))
        zero = Variable(torch.zeros(1))
        zero = zero.to(device)
        max_left = torch.max(m_plus - v_c, zero).view(batch_size, -1)**2
        max_right = torch.max(v_c - m_minus, zero).view(batch_size, -1)**2
        t_c = target
        l_c = t_c * max_left + loss_lambda * (1.0 - t_c) * max_right
        l_c = l_c.sum(dim=1)
        return l_c

    def recloss(self, recon, input_img):
        image = input_img
        batch_size=image.shape[0]
        image=image.view(batch_size,-1)
        recon=recon.view(batch_size,-1)
        error=recon - image
        sq_error=error**2
        recon_error = torch.mean(sq_error, dim=1)
        return recon_error


In [ ]:
model = Net(parameter['N'],dataset_parameter['channel'],parameter['C12'],1,parameter['C11'],parameter['kH'],parameter['kW'],parameter['kH1'],parameter['kW1'],dataset_parameter['num_classes'],parameter['C3'],3,parameter['H'],parameter['W'],parameter['F'],parameter['batch'],parameter['C_FC1'],parameter['C_FC2'],device)

In [ ]:
#evaluation_metrics

class metrics(nn.Module):
    def __init__(self, threshold=0.5):
        super(metrics, self).__init__()
        self.threshold = threshold

    def forward(self, target, input):
        eps = 1e-10
        input_ = (input > self.threshold).data.float()
        target_ = (target > self.threshold).data.float()
        intersection = torch.clamp(input_ * target_, 0, 1)
        union = torch.clamp(input_ + target_, 0, 1)
        if torch.mean(intersection).lt(eps):
            return torch.Tensor([0., 0., 0., 0.])
        else:
            acc = torch.mean((input_ == target_).data.float())
            iou = torch.mean(intersection) / torch.mean(union)
            recall = torch.mean(intersection) / torch.mean(target_)
            precision = torch.mean(intersection) / torch.mean(input_)
            f1_score = 2*recall*precision/(recall+precision)
            return torch.Tensor([acc, f1_score])
metrics=metrics()

In [ ]:
#training_testing_module

def train_one_epoch_net(model, train_dl, learn, device):
    opt = torch.optim.Adam(model.parameters(),lr=learn)
    running_loss_image=0.0
    acc_epoch=0.0
    f1_epoch=0.0
    dice_epoch=0.0
    for a,b in train_dl:
        a=a.float()
        a=torch.nn.functional.interpolate(a, size=(parameter['N']), scale_factor=None, mode='nearest')
        b=b.float()

        ###preparing inputs for model (graph and STFT preparation)
        edge_index = [(edge_idx(a[i])) for i in range(len(a))]
        edges_idx = torch.reshape((torch.cat(edge_index,axis = 0)),[parameter['batch'],np.shape(edge_index[0])[0],np.shape(edge_index[0])[1]])
        edge_weight = [(edge_wt(a[i],edges_idx[i])) for i in range(len(a))]
        edges_wt = torch.reshape((torch.cat(edge_weight,axis = 0)),[parameter['batch'],np.shape(edge_weight[0])[0]])
        p = [torch.abs(torch.stft(a[i],200,return_complex=True)) for i in range(len(a))]
        q = torch.reshape((torch.cat(p,axis = 0)),[parameter['batch'],np.shape(p[0])[0],np.shape(p[0])[1],np.shape(p[0])[2]])
        q=torch.nn.functional.interpolate(q, size=(parameter['H'],parameter['W']), scale_factor=None, mode='nearest')
        graph_tensor = a
        edge_index=edges_idx[0].T
        edge_weight=edges_wt[0]
        batch=parameter['batch']
        alpha = parameter['alpha']
        m_plus = parameter['m_plus']
        m_minus = parameter['m_minus']
        lamda = parameter['lamda']
        a = a.to(device)
        b = b.to(device)
        edge_index = edge_index.to(device)
        edge_weight = edge_weight.to(device)
        batch = torch.tensor(batch).to(device)
        alpha = torch.tensor(alpha).to(device)
        m_plus = torch.tensor(m_plus).to(device)
        m_minus = torch.tensor(m_minus).to(device)
        lamda = torch.tensor(lamda).to(device)
        p = [torch.abs(torch.stft(a[i],200,return_complex=True)) for i in range(len(a))]
        q = torch.reshape((torch.cat(p,axis = 0)),[batch,np.shape(p[0])[0],np.shape(p[0])[1],np.shape(p[0])[2]])
        q=torch.nn.functional.interpolate(q, size=(parameter['H'],parameter['W']), scale_factor=None, mode='nearest')

        pred1,pred3,pred2,label_new=model(graph_tensor,edge_index,edge_weight,q,b)
        loss = model.loss(pred1,pred2,pred3,q,label_new,b,alpha,m_plus,m_minus,lamda)
        opt.zero_grad()
        loss.backward()
        opt.step()
        running_loss_image += loss
        metric=metrics(b,pred1)
        acc_epoch += metric[0]
        f1_epoch += metric[1]
    running_loss_image/=len(train_dl)
    acc_epoch /= len(train_dl)
    f1_epoch /= len(train_dl)
    return model, [acc_epoch,f1_epoch], running_loss_image

def validate_one_epoch_net(model, val_dl, device):
    running_loss_image=0.0
    acc_epoch=0.0
    f1_epoch=0.0
    dice_epoch=0.0
    with torch.no_grad():
      for a,b in train_dl:
        a=a.float()
        a=torch.nn.functional.interpolate(a, size=(parameter['N']), scale_factor=None, mode='nearest')
        b=b.float()


        ###preparing inputs for model (graph and STFT preparation)
        edge_index = [(edge_idx(a[i])) for i in range(len(a))]
        edges_idx = torch.reshape((torch.cat(edge_index,axis = 0)),[parameter['batch'],np.shape(edge_index[0])[0],np.shape(edge_index[0])[1]])
        edge_weight = [(edge_wt(a[i],edges_idx[i])) for i in range(len(a))]
        edges_wt = torch.reshape((torch.cat(edge_weight,axis = 0)),[parameter['batch'],np.shape(edge_weight[0])[0]])
        graph_tensor = a
        edge_index=edges_idx[0].T
        edge_weight=edges_wt[0]
        batch=parameter['batch']
        alpha = parameter['alpha']
        m_plus = parameter['m_plus']
        m_minus = parameter['m_minus']
        lamda = parameter['lamda']
        a = a.to(device)
        b = b.to(device)
        edge_index = edge_index.to(device)
        edge_weight = edge_weight.to(device)
        batch = torch.tensor(batch).to(device)
        alpha = torch.tensor(alpha).to(device)
        m_plus = torch.tensor(m_plus).to(device)
        m_minus = torch.tensor(m_minus).to(device)
        lamda = torch.tensor(lamda).to(device)

        p = [torch.abs(torch.stft(a[i],200,return_complex=True)) for i in range(len(a))]
        q = torch.reshape((torch.cat(p,axis = 0)),[batch,np.shape(p[0])[0],np.shape(p[0])[1],np.shape(p[0])[2]])
        q=torch.nn.functional.interpolate(q, size=(parameter['H'],parameter['W']), scale_factor=None, mode='nearest')

        pred1,pred3,pred2,label_new=model(graph_tensor,edge_index,edge_weight,q,b)
        loss = model.loss(pred1,pred2,pred3,q,label_new,b,alpha,m_plus,m_minus,lamda)
        running_loss_image += loss
        metric=metrics(b,pred1)
        acc_epoch += metric[0]
        f1_epoch += metric[1]
    running_loss_image/=len(train_dl)
    acc_epoch /= len(train_dl)
    f1_epoch /= len(train_dl)
    return [acc_epoch,f1_epoch], running_loss_image

def train_epoches_net(model,train_dl,test_dl,epoches,learn,path,device):
    max_accuracy=0.0
    for i in range(epoches):
        model, metrics_train, loss_train=train_one_epoch_net(model,train_dl,learn,device)
        metrics_test, loss_test=validate_one_epoch_net(model,test_dl,device)
        print('epoch finished' +" " + str(i+1))
        print(f'train_loss: {loss_train:.6f}, train_accuracy: {metrics_train[0]:.6f}, train_f1_score: {metrics_train[1]:.6f}')
        print(f'test_loss: {loss_test:.6f}, test_accuracy: {metrics_test[0]:.6f}, test_f1_score: {metrics_test[1]:.6f}')
        path_final=os.path.join(path,f"epoch{i}_test_loss_{metrics_test[0]:.4f}.pth")
        torch.save(model.state_dict(), path_final)

In [ ]:
#training_testing

%mkdir eeg
train_epoches_net(model,test_dl,test_dl,20,0.001,'/content/eeg',device)